## Chapter 19. Concurrency Models in Python

#### Spinner with Threads

In [12]:
# Example 19-1. spinner_thread.py: the spin and slow functions

import itertools
import time
from threading import Thread, Event

def spin(msg: str, done: Event) -> None:
    for char in itertools.cycle(r'\|/-'):
        status = f'\r{char} {msg}'  # trick for text-mode animation
        print(status, end='', flush=True)
        if done.wait(.1):
            break
    # clear the status line by overwriting with spaces and moving the cursor back to the beginning
    blanks = ' ' * len(status)    
    print(f'\r{blanks}\r', end='')

def slow() -> int:
    time.sleep(3)
    return 42

In [13]:
# Example 19-2. spinner_thread.py: the supervisor and main functions

def supervisor() -> int:
    done = Event()
    spinner = Thread(target=spin, args=('thinking!', done))
    print(f'spinner object: {spinner}')
    spinner.start()
    result = slow()
    done.set()
    spinner.join()
    return result

In [14]:
result = supervisor()
print(f'Answer: {result}')

spinner object: <Thread(Thread-4 (spin), initial)>
Answer: 42  


#### Spinner with Processes

In [20]:
# Example 19-3. spinner_proc.py: only the changed parts are shown; everything else is the same as spinner_thread.py

import itertools
import time
from multiprocessing import Process, Event
from multiprocessing import synchronize

def spin_process(msg: str, done: synchronize.Event) -> None:
# [snip] the rest of spin and slow functions are unchanged from spinner_thread.py
    for char in itertools.cycle(r'\|/-'):
        status = f'\r{char} {msg}'  # trick for text-mode animation
        print(status, end='', flush=True)
        if done.wait(.1):
            break
    # clear the status line by overwriting with spaces and moving the cursor back to the beginning
    blanks = ' ' * len(status)    
    print(f'\r{blanks}\r', end='')

def supervisor_process() -> int:
    done = Event()
    spinner = Process(target=spin, args=('thinking!', done))
    print(f'spinner object: {spinner}')
    spinner.start()
    result = slow()
    done.set()
    spinner.join()
    return result

In [21]:
result = supervisor_process()
print(result)

spinner object: <Process name='Process-4' parent=11428 initial>
42


#### Spinner with Coroutines

In [11]:
# Example 19-5. spinner_async.py: the spin and slow coroutines

import asyncio
import itertools

async def spin(msg: str) -> None:
    for char in itertools.cycle(r'\|/-'):
        status = f'\r{char} {msg}'
        print(status, flush=True, end='')
        try:
            await asyncio.sleep(.1)
        except asyncio.CancelledError:
            break
    blanks = ' ' * len(status)
    print(f'\r{blanks}\r', end='')

async def slow() -> int:
    await asyncio.sleep(3)
    return 42

# Example 19-4. spinner_async.py: the main function and supervisor coroutine
async def supervisor() -> int:
    spinner = asyncio.create_task(spin('thinking!'))
    print(f'spinner object: {spinner}')
    result = await slow()
    spinner.cancel()  # raise CancelledError
    return result

# main should run in a separated file. like 19-4.py

### The Real Impact of the GIL

In [17]:
# Example 19-10. primes.py: an easy to read primality check, from Python’s ProcessPool Executor example

import math

def is_prime(n: int) -> bool:
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False

    root = math.isqrt(n)
    for i in range(3, root + 1, 2):
        if n % i == 0:
            return False
    return True

In [18]:
is_prime(5000111000222021)

True